In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time

# Зчитуємо зображення для детекту і зображення для заповнення

In [16]:
cap = cv2.VideoCapture(0)
img_rafael = cv2.imread('1.jpg')
img_dragon = cv2.imread('2.jpg')

img_warp_rafael = cv2.imread('3.jpg')
img_warp_dragon = cv2.imread('4.jpg')


# Тут обраховуємо ключові точки

In [3]:
hT,wT,cT = img_rafael.shape

sift = cv2.SIFT_create(nfeatures=1000)
kp_rafael, des_rafael = sift.detectAndCompute(img_rafael,None)
kp_dragon, des_dragon = sift.detectAndCompute(img_dragon, None)

# Це потрібно для матчінгу

In [4]:
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

# А ось тут веселість. Починаємо зчитувати кадри з вебки.

In [7]:
frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

In [17]:

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_without = cv2.VideoWriter('out_without1.avi', fourcc = fourcc, fps=30, frameSize=(int(frame_width),int(frame_height)))
out_with = cv2.VideoWriter('outpy_with1.avi', fourcc, 30, (int(frame_width),int(frame_height)))

while True:

    sucess, imgWebcam = cap.read()
    if sucess == False:
        print('Error sucess')
        break

    imgAug = imgWebcam.copy()
    # обраховуємо ключові точки
    kp_webcam, des_webcam = sift.detectAndCompute(imgWebcam, None)
    try:
        # тут шукаємо співпадіння
        matches_rafael = flann.knnMatch(des_rafael, des_webcam, k=2)
        matches_dragon = flann.knnMatch(des_dragon, des_webcam, k=2)
    except:
        cv2.imshow('maskNew', imgWebcam)
        out_without.write(imgWebcam)
        continue

    good_rafael =[]
    good_dragon = []

    # вибираємо особливі
    for m,n in matches_rafael:
        if m.distance < 0.75*n.distance:
            good_rafael.append(m)

    for m,n in matches_dragon:
        if m.distance < 0.75*n.distance:
            good_dragon.append(m)

    # ось ту відбувається примітивна класифікація
    if len(good_rafael) > len(good_dragon) and len(good_rafael) > 50:
        try:
            # вибираємо точки
            srcPts = np.float32([kp_rafael[m.queryIdx].pt for m in good_rafael]).reshape(-1, 1, 2)
            dstPts = np.float32([kp_webcam[m.trainIdx].pt for m in good_rafael]).reshape(-1, 1, 2)
            # шукаємо матрицю для перетворення зображення в ту саму форму
            matrix, mask = cv2.findHomography(srcPts,dstPts,cv2.RANSAC,5)

            # ця штука потрібна для маски. Так би мовити готовимо холст
            pts = np.float32([[0,0],[0,hT],[wT,hT],[wT,0]]).reshape(-1,1,2)
            dst = cv2.perspectiveTransform(pts,matrix)

            # готуємо нове зображення для заповнення
            imgWarp = cv2.warpPerspective(img_warp_rafael, matrix, (imgWebcam.shape[1],imgWebcam.shape[0]))

            # створюємо маску для вирізу старого зображення і вставки нового
            maskNew = np.zeros((imgWebcam.shape[0],imgWebcam.shape[1]),np.uint8)
            cv2.fillPoly(maskNew,[np.int32(dst)],(255,255,255))
            maskInv = cv2.bitwise_not(maskNew)
            # вирізаємо
            imgAug = cv2.bitwise_and(imgAug, imgAug, mask = maskInv)
            # вставляємо
            imgAug = cv2.bitwise_or(imgWarp,imgAug)

            # демонструємо перетворене зображення
            cv2.imshow('maskNew', imgAug)
            out_with.write(imgAug)
            out_without.write(imgWebcam)

            # для виходу q
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        except:
            cv2.imshow('maskNew', imgWebcam)
            out_without.write(imgWebcam)
            continue
    elif len(good_rafael) < len(good_dragon) and len(good_dragon) > 50:
        try:
            detection = True
            # вибираємо точки
            srcPts = np.float32([kp_dragon[m.queryIdx].pt for m in good_dragon]).reshape(-1, 1, 2)
            dstPts = np.float32([kp_webcam[m.trainIdx].pt for m in good_dragon]).reshape(-1, 1, 2)
            # шукаємо матрицю для перетворення зображення в ту саму форму
            matrix, mask = cv2.findHomography(srcPts,dstPts,cv2.RANSAC,5)

            # ця штука потрібна для маски. Так би мовити готовимо холст
            pts = np.float32([[0,0],[0,hT],[wT,hT],[wT,0]]).reshape(-1,1,2)
            dst = cv2.perspectiveTransform(pts,matrix)

            # готуємо нове зображення для заповнення
            imgWarp = cv2.warpPerspective(img_warp_dragon, matrix, (imgWebcam.shape[1],imgWebcam.shape[0]))

            # створюємо маску для вирізу старого зображення і вставки нового
            maskNew = np.zeros((imgWebcam.shape[0],imgWebcam.shape[1]),np.uint8)
            cv2.fillPoly(maskNew,[np.int32(dst)],(255,255,255))
            maskInv = cv2.bitwise_not(maskNew)
            # вирізаємо
            imgAug = cv2.bitwise_and(imgAug, imgAug, mask = maskInv)
            # вставляємо
            imgAug = cv2.bitwise_or(imgWarp,imgAug)
            # демонструємо перетворене зображення
            cv2.imshow('maskNew', imgAug)
            out_with.write(imgAug)
            out_without.write(imgWebcam)
            # для виходу q
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        except:
            cv2.imshow('maskNew', imgWebcam)
            out_without.write(imgWebcam)
            continue
    else:
        cv2.imshow('maskNew', imgWebcam)
        out_with.write(imgAug)
        out_without.write(imgWebcam)
        # для виходу q
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


In [18]:
cap.release()
out_without.release()
out_with.release()
cv2.destroyAllWindows()